<a href="https://colab.research.google.com/github/anosharahim/capstone/blob/master/emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [24]:
import pandas as pd
import os
import zipfile

In [38]:
#zipped_path = '/content/drive/MyDrive/Capstone/datasets/archive.zip'
#target_path = "/content/drive/MyDrive/Capstone/FER2013'
#unzip and extract files to gdrive
#!unzip zipped_path -d target_path

In [52]:
train_path = '/content/drive/MyDrive/Capstone/FER2013/train/'
test_path = '/content/drive/MyDrive/Capstone/FER2013/test/'

train_data = []
train_label = []
test_data = []
test_label = []

num_expressions = 6
labels = ['angry', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']


In [57]:
categories = os.listdir(train_path)
categories

['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']